In [1]:
from IPython.display import display, HTML, clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import prodimopy.read as pread
import pickle
import numpy as np
import pandas as pd
import time
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

In [3]:
import glob

wlrange = [5, 28]
species = ['C2H2_H', 'CH4_H', 'CO', 'CO2_H', 'HCN_H', 'NH3_H', 'NO_H', 'OH', 'o-H2', 'o-H2O', 'p-H2', 'p-H2O']
ions = ['Ar+', 'Ar++', 'Fe+', 'Ne+', 'Ne++', 'S']
values = np.arange(-0.5, 0.51, 0.25)
data = []
start_time = time.time()
i = 1
for C in values:
    for O in values:
        start_time = time.time()
        
        model_data = []
        
        path = 'models/C_' + str(C) + '_O_' + str(O) + '_CO_*'
        matching_file = glob.glob(path)
        
        CO = matching_file[0].split('CO_')[-1]
        model_name = matching_file[0][7:]
        model_data.append(model_name)
        model_data.append(str(C))
        model_data.append(str(O))
        model_data.append(str(CO))

        model = pread.read_prodimo(matching_file[0], filenameFLiTs='specFLiTs1.out')

        wl, flux, flux_cont = model.FLiTsSpec.convolve(3000, contReturn=True)
        flux -= flux_cont
        model_data.append(wl)
        model_data.append(flux)

        for specie in species:
            model_data.append(model.gen_specFromLineEstimates(ident=specie, wlrange=wlrange, noCont=True, unit='Jy')[1])
        
        total_flux_ion = None
        for ion in ions:
            wl, flux_ion = model.gen_specFromLineEstimates(ident=ion, wlrange=wlrange, noCont=True, unit='Jy')
            if total_flux_ion is None:
                total_flux_ion = flux_ion
            else:
                total_flux_ion += flux_ion
        model_data.append(total_flux_ion)
        model_data.append(wl)
        
        for flits in ['atomic', 'CO', 'CO2', 'H2O', 'HCN', 'NH3', 'NO', 'O2', 'OH']:
            model_flits = pread.read_FLiTs(f'./FLiTs/{flits}/'+model, filename=f'specFLiTs1_{flits}.out')
            model_data.append(model_flits.flux-model_flits.flux_cont)
            model_data.append(model_flits.wl)
        
        data.append(model_data)
        
        end_time = time.time()
        clear_output()
        current_time = end_time - start_time
        print(f'{i}/25; {int(current_time // 60)}:{int(current_time % 60)}')
        i += 1


READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\ProDiMo.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\Species.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\FlineEstimates.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\Elements.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\dust_opac.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\dust_sigmaa.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\StarSpectrum.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\SED.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\SEDana.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\image.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\specFLiTs1.out  ...
READ: Reading File:  models\C_-0.5_O_-0.5_CO_0.45\Parameter.out  ...
INFO: Reading time:  27.14 s
 
INFO: convolve FLiTs spectrum ... 
INFO: gen_specFromLineEstimates: build spectrum for 26457 lines ...
INFO: gen_specFromLineEstimates: convolve spe

KeyboardInterrupt: 

In [97]:
df = pd.DataFrame(data,
                  columns=['Model Name', 'C Value', 'O Value', 'CO Value', 'FLiTs Wavelength', 'FLiTs Flux',
                           'ProDiMo C2H2_H',
                           'ProDiMo CH4_H', 'ProDiMo CO', 'ProDiMo CO2_H', 'ProDiMo HCN_H', 'ProDiMo NH3_H',
                           'ProDiMo NO_H', 'ProDiMo OH', 'ProDiMo o-H2', 'ProDiMo o-H2O', 'ProDiMo p-H2',
                           'ProDiMo p-H2O', 'ProDiMo atomic', 'ProDiMo Wavelength', 'FLiTs atomic Flux',
                           'FLiTs atomic Wavelength', 'FLiTs CO Flux', 'FLiTs CO Wavelength', 'FLiTs CO2 Flux',
                           'FLiTs CO2 Wavelength', 'FLiTs H2O Flux', 'FLiTs H2O Wavelength', 'FLiTs HCN Flux',
                           'FLiTs HCN Wavelength', 'FLiTs NH3 Flux', 'FLiTs NH3 Wavelength', 'FLiTs NO Flux',
                           'FLiTs NO Wavelength', 'FLiTs O2 Flux', 'FLiTs O2 Wavelength', 'FLiTs OH Flux',
                           'FLiTs OH Wavelength'])

with open('ModelData.pkl', 'wb') as f:
    pickle.dump(df, f)
